In [1]:
!pip install PyTDC
!pip install rdkit

In [2]:
import pandas as pd
import numpy as np
from tdc.multi_pred import DTI
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.Chem import EState
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import GraphDescriptors
from rdkit.Chem import Crippen
from rdkit.Chem import QED

Note: BindingDB is the collection of many assays. Since different assays use different metrics, TDC separates them as separate datasets. Specifically, it has four datasets with Kd, IC50, Ki as the units.

In [3]:
data = DTI(name = 'BindingDB_Kd')
# data = DTI(name = 'BindingDB_IC50')
# data = DTI(name = 'BindingDB_Ki')
split = data.get_split()

Found local copy...
Loading...
Done!


Tips: Transforming to log-scale pIC50, pKi, and pKd can usually lead to more stable training. You can achieve this transformation via here. Checkout the data processing page for binarization, label distribution visualization, edge list/DGL/PyTorch graph transformation.

In [4]:
data.convert_to_log(form = 'binding')
# convert back: data.convert_from_log(form = 'binding')

To log space...


Note: Many DTI pairs have same sequence information but different binding affinities due to different experimental assays. To harmonize them, you can use the below function to retrieve either the maximum affinity or the mean for the duplicated pair:



In [5]:
data.harmonize_affinities(mode = 'max_affinity')
# data.harmonize_affinities(mode = 'mean')

The scale is converted to log scale, so we will take the maximum!
The original data has been updated!


,Drug_ID,Drug,Target_ID,Target,Y
0,51.0,O=C(O)CCC(=O)C(=O)O,Q9GZT9,MANDSGGPGGPSPSERDRQYCELCGKMENLLRCSRCRSSFYCCKEH...,6.045709
1,187.0,CC(=O)OCC[N+](C)(C)C,P11229,MNTSAPPAVSPNITVLAPGKGPWQVAFIGITTGLLSLATVTGNLLV...,4.759998
2,187.0,CC(=O)OCC[N+](C)(C)C,P58154,MRRNIFCLACLWIVQACLSLDRADILYNIRQTSRPDVIPTQRDRPV...,6.339040
3,237.0,CCN(CC)CCCC(C)Nc1c2ccc(Cl)cc2nc2ccc(OC)cc12,P02752,MLRFAITLFAVITSSTCQQYGCLEGDTHKANPSPEPNMHECTLYSE...,6.578232
4,242.0,O=C([O-])c1ccccc1,P14920,MRVVVIGAGVIGLSTALCIHERYHSVLQPLDIKVYADRFTPLTTTD...,5.657558
...,...,...,...,...,...
42231,138805907.0,COc1nc2ccc([C@@](O)(c3ccc(C(F)(F)F)nc3)c3cncn3...,P51449,MDRAPQRQHRASRELLAAKKTHTSQIEVIPCKICGDKSSGIHYGVI...,8.494850
42232,138805908.0,COc1nc2ccc([C@](O)(c3ccnc(C(F)(F)F)c3)c3cncn3C...,P51449,MDRAPQRQHRASRELLAAKKTHTSQIEVIPCKICGDKSSGIHYGVI...,5.031512
42233,138805909.0,COc1nc2ccc([C@@](O)(c3ccnc(C(F)(F)F)c3)c3cncn3...,P51449,MDRAPQRQHRASRELLAAKKTHTSQIEVIPCKICGDKSSGIHYGVI...,8.443697
42234,138805910.0,COc1nc2ccc([C@](O)(c3ccc(Cl)cc3)c3cncn3C)cc2c(...,P51449,MDRAPQRQHRASRELLAAKKTHTSQIEVIPCKICGDKSSGIHYGVI...,6.070530


Feature Generation for Drug Molecules (using RDKit)

In [6]:
# Molecular weight
def MW(drug):
  m = Chem.MolFromSmiles(drug)
  # print("MW:", Descriptors.MolWt(m))
  return Descriptors.MolWt(m)

# Mean electrotopological state
def Ms(drug):
  m = Chem.MolFromSmiles(drug)
  estates = EState.EStateIndices(m)
  # print("Ms:", np.mean(estates))
  return np.mean(estates)

# number of 5 membered rings
def nR05(drug):
  m = Chem.MolFromSmiles(drug)
  # print("nR05:", len(Chem.GetSSSR(m)))
  return len(Chem.GetSSSR(m))

Feature Aggregation (Drug Molecule)

In [7]:
def feature_gen(drug_sequences):
  features = []
  for drug in drug_sequences:
      MW_out = MW(drug)
      Ms_out = Ms(drug)
      nR05_out = nR05(drug)
      features.append([MW_out, Ms_out, nR05_out])
  features_arr = np.array(features)
  return features_arr

Feature Normalization (Drug Molecule)

In [8]:
from sklearn.preprocessing import StandardScaler

train_features_arr = feature_gen(split['train']['Drug'])
test_features_arr = feature_gen(split['test']['Drug'])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(train_features_arr)
test_X_scaled = scaler.transform(test_features_arr)